In [1]:
from google.colab import auth
import os
auth.authenticate_user()

!gsutil ls gs://tlm-testing/
os.environ['COLAB_SKIP_TPU_AUTH'] = '1'

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!mkdir meta-data
!gsutil cp gs://tlm-testing/auto500k/*.30k meta-data/
!gsutil cp gs://tlm-testing/auto500k/train/md.yml meta-data/train_md.yml
!gsutil cp gs://tlm-testing/auto500k/valid/md.yml meta-data/valid_md.yml

In [ ]:
!pip install tensorflow
!pip install fastBPE

In [ ]:
!git clone -b feature/test-tpu https://github.com/wenshuoliu/baseline.git
!cd baseline/layers/ && pip install -e .
!cd baseline/ && pip install -e .

In [ ]:
!echo $COLAB_TPU_ADDR

In [ ]:
!python baseline/api-examples/pretrain_tlm_tf.py --basedir gs://tlm-testing/job-dir --subword_model_file meta-data/codes.30k --subword_vocab_file meta-data/vocab.30k --dataset_key auto \\
--train_dir gs://tlm-testing/auto500k/train --valid_dir gs://tlm-testing/auto500k/valid --train_md meta-data/train_md.yml --valid_md meta-data/valid_md.yml \\
--distribute tpu --tpu_ep colab --file_type tfrecord --rpr_k 3 --epochs 2 --warmup_steps 1000 --optim adamw --lr 4.e-4 --weight_decay 1.e-3 --clip 1.0 \\
--nctx 256 --embed_type default --num_layers 8 --num_heads 1 --d_k 64 --d_model 512 --d_ff 2048 --batch_size 512 --saves_per_epoch 2 \\
--windowed_ra 1 --rpr_value_on 0 --npz 1